In [4]:
import pandas as pd
import math
import os
from datetime import datetime
import string
import xlrd
from openpyxl import load_workbook
import unicodedata
import re

In [98]:
df = pd.DataFrame()
punctuations = string.punctuation
digits = string.digits
for company in os.listdir('../Data'):
    root = f'../Data/{company}/10-Q'
    entry = {'Company':company}
    for file in os.listdir(root):
#         file = re.sub('[/!@#$%^&*~]+','',file)
        possible_sheet_names = ['Condensed Consolidated Income S','Consolidated Statements of Oper',
                                'Condensed Statements of Operat','CONDENSED CONSOLIDATED STATEMEN',
                                'Condensed Consolidated Statemen','CONSOLIDATED STATEMENTS OF OPER'
                                'Unaudited Consolidated Statemen','Unaudited Condensed Consolida_3'
                                'STATEMENTS_OF_OPERATIONS','Statements of Operations (Unaud',
                               'Consolidated_Statements_of_Ope','Consolidated Statements Of Oper',
                               'Condensed Consolidated Statem_3']
        file_path = root + '/' + file
#         print(file_path)
        try:
            sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[0])
        except:
            try:
                sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[1])
            except:
                try:
                    sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[2])
                except:
                    try:
                        sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[3])
                    except:
                        try:
                            sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[4])
                        except:
                            try:
                                sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[5])
                            except:
                                try:
                                    sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[6])
                                    print(sheet)
                                except:
                                    try:
                                        sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[7])
                                    except:
                                        try:
                                            sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[8])
                                        except:
                                            try:
                                                sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[9])
                                            except:
                                                try:
                                                    sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[10])
                                                except:
                                                    try:
                                                        sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[11])
                                                    except:
                                                        try:
                                                            sheet = pd.read_excel(file_path,sheet_name=possible_sheet_names[12])
                                                        except:
                                                            pass
        len_col = sheet.shape[0]
        # sheet.dropna(thresh=int(len_col*0.75),axis=1,inplace = True)
        sheet.iloc[:,0] = [str(index).lower() for index in list(sheet.iloc[:,0])]
        sheet.iloc[:,0] = sheet.iloc[:,0].apply(lambda x: str(x).translate(str.maketrans('','',punctuations)))
        sheet.iloc[:,0] = sheet.iloc[:,0].apply(lambda x: str(x).translate(str.maketrans('','',digits)))
        sheet.iloc[:,0] = sheet.iloc[:,0].apply(lambda x: unicodedata.normalize('NFKD',x).strip())
        possible_keys = ['revenue','net revenue','nan','revenues','net revenues','total revenue','total revenues',
                        'total net revenue','total net revenues','total operating revenue','total operating revenues',
                        'operating revenue','total subscription revenue','total subscription revenues',
                        'revenues net']

        revenue = sheet[sheet.iloc[:,0].isin(possible_keys)]
        revenue = revenue.dropna(axis = 1,thresh = int(0.2*revenue.shape[0]))
        columns = list(revenue.columns.str.lower())
        revenue.columns = columns
        try:
            index = columns.index('3 months ended')
        except:
            try:
                index = columns.index('2 months ended')
            except:
                index = 1
                    
                    
        try:
            revenue = revenue.iloc[:,[0,index,index+1]].dropna()[:2]
        except:
            try:
                revenue = revenue.iloc[:,[0,index]].dropna()[:2]
            except:
                pass
        try:
            date_1 = datetime.strptime(revenue.iloc[0,1],'%b. %d, %Y')
            date_1 = date_1.strftime('%b %Y')
            date_2 = datetime.strptime(revenue.iloc[0,2],'%b. %d, %Y')
            date_2 = date_2.strftime('%b %Y')

        except:
            try:
                date_1 = datetime.strptime(revenue.iloc[0,1],'%b %d, %Y')
                date_1 = date_1.strftime('%b %Y')
                date_2 = datetime.strptime(revenue.iloc[0,2],'%b %d, %Y')
                date_2 = date_2.strftime('%b %Y')
            except:
                try:
                    date_1 = datetime.strptime(revenue.columns[1],'%b. %d, %Y')
                    date_1 = date_1.strftime('%b %Y')
                    date_2 = datetime.strptime(revenue.columns[2],'%b. %d, %Y')
                    date_2 = date_2.strftime('%b %Y')
                except:
                    pass
                    
        try:    
            revenue.iloc[1,1:] = revenue.iloc[1,1:].apply(abs)
            revenue.iloc[0,1:] = revenue.iloc[0,1:].apply(abs)
        except:
            pass
        try:
            if 'thousands' in revenue.columns[0]:
                entry[date_1] = int(float(revenue.iloc[1,1])*1000)
                entry[date_2] = int(float(revenue.iloc[1,2])*1000)
            elif 'millions' in  revenue.columns[0]:
                entry[date_1] = int(float(revenue.iloc[1,1])*1000000)
                entry[date_2] = int(float(revenue.iloc[1,2])*1000000)
            else:
                entry[date_1] = int(float(revenue.iloc[1,1]))
                entry[date_2] = int(float(revenue.iloc[1,2]))
        except:
            pass

    df = df.append(entry,ignore_index=True)

In [99]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
empty = df[df.iloc[:,1:].isnull().sum(axis = 1) == df.shape[1]-1]

In [102]:
len(empty)

78

In [100]:
bad_companies = []
for company in empty['Company']:
    q10 = len(os.listdir(f'../Data/{company}/10-Q'))
    if q10 != 0:
        bad_companies.append(company)
bad_companies        

['ALLDIGITAL HOLDINGS INC',
 'BLACK BOX CORP',
 'BSQUARE CORP',
 'CLONE ALGO TECHNOLOGIES INC',
 'SECUREWORKS CORP',
 'SMARTMETRIC INC']

In [103]:
df.to_csv('../Revenue.csv')

In [3]:
df = pd.read_csv('../Revenue.csv').drop(columns=['Unnamed: 0'])
df.set_index('Company',inplace = True)
df.head()

,Jun 2017,Jun 2018,Jun 2019,Jun 2020,Jun 2021,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,...,Jan 2018,Jan 2019,Jan 2020,Jan 2021,Jun 2016,Sep 2016,Mar 2014,Dec 2013,Jun 2013,Mar 2013
Company,,,,,,,,,,,,,,,,,,,,,
2U INC,64995000.0,97423000.0,135461000.0,182687000.0,237209000.0,64829000.0,92288000.0,122234000.0,175479000.0,232473000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30DC INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3D PIONEER SYSTEMS INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8X8 INC,69098000.0,83225000.0,96675000.0,121807000.0,148327000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10 NETWORKS INC,53973000.0,60713000.0,49189000.0,52500000.0,59168000.0,63934000.0,49183000.0,50290000.0,53764000.0,54843000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
idx = pd.to_datetime(df.columns,errors = 'coerce',format = '%b %Y').argsort()
df = df.iloc[:,idx]
df.to_csv('../Revenue.csv')

In [20]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
df

,Sep 2012,Mar 2013,Jun 2013,Sep 2013,Dec 2013,Mar 2014,May 2014,Jun 2014,Aug 2014,Sep 2014,Oct 2014,Nov 2014,Dec 2014,Mar 2015,May 2015,Jun 2015,Aug 2015,Sep 2015,Oct 2015,Nov 2015,Dec 2015,Mar 2016,May 2016,Jun 2016,Aug 2016,Sep 2016,Nov 2016,Dec 2016,Jan 2017,Mar 2017,Apr 2017,May 2017,Jun 2017,Jul 2017,Aug 2017,Sep 2017,Oct 2017,Nov 2017,Dec 2017,Jan 2018,Mar 2018,Apr 2018,Jun 2018,Jul 2018,Aug 2018,Sep 2018,Oct 2018,Dec 2018,Jan 2019,Mar 2019,Apr 2019,May 2019,Jun 2019,Jul 2019,Aug 2019,Sep 2019,Oct 2019,Dec 2019,Jan 2020,Feb 2020,Mar 2020,Apr 2020,May 2020,Jun 2020,Jul 2020,Aug 2020,Sep 2020,Oct 2020,Dec 2020,Jan 2021,Mar 2021,Apr 2021,Jun 2021,Jul 2021,Sep 2021,Oct 2021
Company,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2U INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64829000.0,NaN,NaN,6.499500e+07,NaN,NaN,7.025000e+07,NaN,NaN,NaN,NaN,92288000.0,NaN,9.742300e+07,NaN,NaN,1.069630e+08,NaN,NaN,NaN,1.222340e+08,NaN,NaN,1.354610e+08,NaN,NaN,153798000.0,NaN,NaN,NaN,NaN,1.754790e+08,NaN,NaN,1.826870e+08,NaN,NaN,2.010730e+08,NaN,NaN,NaN,2.324730e+08,NaN,2.372090e+08,NaN,2.323760e+08,NaN
30DC INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,603244.0,NaN,NaN,49583.0,43689.0,NaN,NaN,NaN,67905.0,NaN,NaN,227855.0,118618.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.010730e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.323760e+08,NaN
3D PIONEER SYSTEMS INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8X8 INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43689.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118618.0,NaN,NaN,NaN,NaN,NaN,6.367600e+07,NaN,NaN,NaN,NaN,6.909800e+07,NaN,NaN,NaN,NaN,NaN,7.557500e+07,NaN,NaN,NaN,8.322500e+07,NaN,NaN,8.568200e+07,NaN,89912000.0,NaN,NaN,NaN,NaN,9.667500e+07,NaN,NaN,109517000.0,NaN,1.185670e+08,NaN,NaN,NaN,NaN,NaN,1.218070e+08,NaN,NaN,1.291330e+08,NaN,1.366850e+08,NaN,NaN,NaN,1.483270e+08,NaN,1.515570e+08,NaN
A10 NETWORKS INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63934000.0,NaN,NaN,5.397300e+07,NaN,NaN,6.200500e+07,NaN,NaN,NaN,NaN,49183000.0,NaN,6.071300e+07,NaN,NaN,6.050200e+07,NaN,NaN,NaN,5.029000e+07,NaN,NaN,4.918900e+07,NaN,NaN,52833000.0,NaN,NaN,NaN,NaN,5.376400e+07,NaN,NaN,5.250000e+07,NaN,NaN,5.660800e+07,NaN,NaN,NaN,5.484300e+07,NaN,5.916800e+07,NaN,6.536000e+07,NaN
ACI WORLDWIDE INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,231462000.0,NaN,NaN,2.405990e+08,NaN,NaN,2.257350e+08,NaN,NaN,NaN,NaN,209310000.0,NaN,2.349950e+08,NaN,NaN,2.455250e+08,NaN,NaN,NaN,2.058550e+08,NaN,NaN,2.976180e+08,NaN,NaN,354901000.0,NaN,NaN,NaN,NaN,2.914850e+08,NaN,NaN,2.999100e+08,NaN,NaN,3.158830e+08,NaN,NaN,NaN,2.851860e+08,NaN,3.016690e+08,NaN,3.169140e+08,NaN
ACTUA CORP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.349950e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.976180e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADEXA INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADOBE INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,